# Declare Constants

In [1]:
# Declare Constants
BATCH_SIZE = 128
GPU = 0
EPOCHS = 10
NUM_CLASSES = 10
DATASET_NAME = 'mnist'

# Import Dataset from Tensorflow Datasets

In [2]:
"""
Since by default tensorflow uses up all available memory in the GPU,
We will set it to allow memory growth.
Also, here we check if the enterred data is correct, before proceeding
""" 

import os

# Enable/Disable GPU and Force Allow Memory Growth
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


import tensorflow as tf
import tensorflow_datasets as tfds

# If GPU is available, Configure Tensorflow to allow memory growth
if tf.config.experimental.list_physical_devices('GPU'):
    # For TF 1x
    if int(tf.__version__.split('.')[0]) == 1:
        from tensorflow.keras.backend import set_session
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.Session(config=config)
        set_session(session)

    # For TF 2x
    else:
        gpu_devices = tf.config.experimental.list_physical_devices(
            'GPU'
        )
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)


# Verify Compaibility of Version of TFDS and Check if Dataset exists
assert int(tfds.__version__.split('.')[0]) == 3, "Please download Tensorflow Datasets v3.x"
assert DATASET_NAME in tfds.list_builders(), "The Entered Dataset is not found, please download it manually"

print(f"Tensorflow Version: {tf.__version__}")
print(f"Tensorflow Datasets Version: {tfds.__version__}")

Tensorflow Version: 2.0.1
Tensorflow Datasets Version: 3.1.0


In [3]:
"""
Load the dataset and split it into train, validation and test set
Since this does not come with a vaildation split, we are creating one
In this case, we are doing a 80:20 split for train and validation set
And we are keeping the test set untouched
Even though the dataset is small, let's follow some rules and load at a given batch_size
"""
dataset, info = tfds.load(
    'mnist',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    data_dir='data',
    with_info=True,
    as_supervised=True,
    batch_size=BATCH_SIZE
)
print("Dataset Info:", info)

# Seperate the train and test data
train_data, val_data, test_data = dataset
train_data, val_data = train_data.repeat(), val_data.repeat()

Dataset Info: tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)



In [4]:
# Verify the Number of training, validation and testing samples
print(f"Number of Training Samples: {info.splits['train[:80%]'].num_examples}")
print(f"Number of Validation Samples: {info.splits['train[80%:]'].num_examples}")
print(f"Number of Testing Samples: {info.splits['test'].num_examples}")

Number of Training Samples: 48000
Number of Validation Samples: 12000
Number of Testing Samples: 10000


# Image Augmentation

In [5]:
# Tensorflow has a very bad implementation of the datasets module.
# Hence, we will be creating a class for Image Augmentation.
# -- More Augmnetaion TO BE ADDED -- #

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., tf.one_hot(indices=label, depth=NUM_CLASSES)

In [6]:
# Verify that we are getting the correct image shape from the generator
generated_batch = tfds.as_numpy(train_data.map(normalize_img)).__next__()
print("Batch of X:", generated_batch[0].shape)
print("Batch of y:", generated_batch[1].shape)

Batch of X: (128, 28, 28, 1)
Batch of y: (128, 10)


# Model Architecture

In [7]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [8]:
# Compile the the model with various metrics
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[
        tf.metrics.CategoricalAccuracy(), # Alternatively, 'accuracy' can also be used
        tf.metrics.Precision(),
        tf.metrics.Recall(),
        tf.metrics.FalseNegatives(),
        tf.metrics.FalsePositives()
    ],
)

# Since the data is being loaded from a generator, we use that to train the model
model.fit_generator(
    train_data.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE),
    epochs=EPOCHS,
    steps_per_epoch=750,
    validation_data=val_data.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE),
    validation_steps=187
)

Epoch 1/10
750/750 [==============================] - 19s 25ms/step - loss: 0.1863 - categorical_accuracy: 0.9439 - precision: 0.9636 - recall: 0.9294 - false_negatives: 6781.0000 - false_positives: 3374.0000 - val_loss: 0.0475 - val_categorical_accuracy: 0.9862 - val_precision: 0.9882 - val_recall: 0.9843 - val_false_negatives: 376.0000 - val_false_positives: 280.0000
Epoch 2/10
750/750 [==============================] - 16s 22ms/step - loss: 0.0653 - categorical_accuracy: 0.9802 - precision: 0.9833 - recall: 0.9779 - false_negatives: 2123.0000 - false_positives: 1598.0000 - val_loss: 0.0402 - val_categorical_accuracy: 0.9879 - val_precision: 0.9887 - val_recall: 0.9874 - val_false_negatives: 302.0000 - val_false_positives: 270.0000
Epoch 3/10
750/750 [==============================] - 16s 22ms/step - loss: 0.0458 - categorical_accuracy: 0.9856 - precision: 0.9873 - recall: 0.9838 - false_negatives: 1551.0000 - false_positives: 1215.0000 - val_loss: 0.0360 - val_categorical_accuracy: 

In [9]:
results = model.evaluate(test_data.map(normalize_img), use_multiprocessing=True, workers=4)

79/79 [==============================] - 1s 8ms/step - loss: 0.0314 - categorical_accuracy: 0.9925 - precision: 0.9925 - recall: 0.9921 - false_negatives: 79.0000 - false_positives: 75.0000
